In [1]:
import pandas as pd
import numpy as np
#data visualization
import matplotlib.pyplot as plt
import librosa

from scipy.stats import kurtosis
from scipy.stats import skew

pd.options.display.precision = 10

## Umgebungsvariablen

In [2]:
feature_data_path = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/earthquakeFeatures_ext.csv'''
cat_boost_model = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/catboost.dump'''

# Feature laden

In [3]:
train_data = pd.read_csv(feature_data_path, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [4]:
train_data.iloc[:,1:93].head()

,mean,std,kurt,skew,min,max,1%q,5%q,25%q,75%q,...,mfcc_kurt,mfcc_skew,mfcc_min,mfcc_max,mfcc_1%q,mfcc_cent_5%q,mfcc_25%q,mfcc_75%q,mfcc_95%q,mfcc_99%q
0,4.8841133333,5.1011061306,33.6624812935,-0.0240611666,-98.0,104.0,-8.0,-2.0,3.0,7.0,...,1.7926250934,1.1048759751,-92.1321830370,-26.1769160053,-90.0217627353,-86.1592492713,-79.7193050066,-65.7660047365,-50.0668287365,-32.7776445855
1,4.8460000000,5.3572687695,107.6590450983,0.3925487681,-154.0,181.0,-8.0,-2.0,3.0,7.0,...,8.0171198003,2.1118007117,-94.6178824145,8.7770193809,-91.3045186191,-86.4059601901,-80.3792653428,-66.6538662214,-49.0992852678,-22.1753170523
2,4.8111600000,5.3222446671,110.4614763269,0.4131461554,-154.0,181.0,-8.0,-2.0,2.0,7.0,...,8.4097814588,2.1560211118,-93.5586801788,8.4034865258,-90.8004274661,-87.0472882567,-80.2364598913,-66.9781009956,-49.1871208009,-21.8952768726
3,4.7970666667,5.4298018331,102.2839738481,0.3948502125,-154.0,181.0,-8.0,-2.0,2.0,7.0,...,7.6028397846,2.0502703394,-91.5577131484,5.1744835678,-89.8584868555,-86.7583032378,-79.7758637117,-65.7594193299,-46.1734484492,-25.5316359156
4,4.7963466667,6.2361343193,112.5711765807,0.4967878159,-154.0,181.0,-9.0,-2.0,2.0,7.0,...,8.7301748746,2.3213224679,-97.9336550547,14.4407606366,-93.0912226709,-85.1069205537,-79.4264194550,-65.0716703608,-44.3139940022,-8.9330467796


## Testdaten vorbereiten

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    features.iloc[:,1:93], features.iloc[:,-1], random_state=0, test_size=0.25)

# Create Model Workbench

## Imports

In [8]:
# Scaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Model selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures

# Modell
from sklearn.ensemble import GradientBoostingRegressor
from catboost import CatBoostRegressor, Pool
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR

# CatBoost

https://github.com/catboost/tutorials/blob/master/python_tutorial.ipynb

## Erster Versuch

In [6]:
#Model #1 - Catboost

train_pool = Pool(X_train, y_train)
m = CatBoostRegressor(iterations=10000, learning_rate=1, loss_function='MAE', boosting_type='Ordered',task_type='GPU')
m.fit(train_pool, silent=True)
print(m.best_score_)

{'learn': {'MAE': 0.519663745777027}}


In [7]:
#Model #1 - Catboost

train_pool = Pool(X_train, y_train)
m = CatBoostRegressor(iterations=10000, learning_rate=1, loss_function='MAE', boosting_type='Ordered')
m.fit(train_pool, silent=True)
print(m.best_score_)

{'learn': {'MAE': 0.5413089612597032}}


Save CatBoost model

In [25]:
#https://catboost.ai/docs/concepts/python-reference_catboost_save_model.html
m.save_model(cat_boost_path, 
           format="cbm", 
           export_parameters=None,
           pool=None)

In [ ]:
model = CatBoostClassifier()
model.load_model(cat_boost_path);

## Feature Importances CatBoost


In [170]:
feature_importances = m.get_feature_importance(train_pool)
feature_names = X_train.columns
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

skew: 3.7327485058842353
zero_crossings: 2.6910612666765195
chroma_stft_95%q: 2.492919097851608
chroma_stft_max: 2.433370716668
spec_bw_max: 2.309247317085658
chroma_stft_75%q: 2.2380370796342484
mfcc_kurt: 2.212971303619402
chroma_stft_99%q: 2.2002330843379094
mfcc_skew: 2.167079335930965
rms_kurt: 2.1335816699215036
min: 2.0465433937015933
chroma_stft_25%q: 2.0404246925114564
spec_cent_max: 2.024123956689253
spec_bw_99%q: 1.9896295759118838
rms_skew: 1.9504872156983881
kurt: 1.9240578966757738
max: 1.8776868053134221
chroma_stft_mean: 1.7746410969446156
chroma_stft_skew: 1.7474203251129428
spec_cent_75%q: 1.7346653567125268
absMax: 1.7147830785704512
spec_bw_95%q: 1.6481040240890352
spec_cent_95%q: 1.632193162272066
chroma_stft_kurt: 1.6308604302491063
chroma_stft_cent_5%q: 1.6195931150780902
mfcc_min: 1.5905301411387578
mfcc_max: 1.5582226588007155
spec_bw_75%q: 1.5080148235854145
rolloff_min: 1.4766048108376157
chroma_stft_1%q: 1.410350911652337
rms_std: 1.3869328563784704
spec_cen

# Hyperparameter Optimierung

load model

In [ ]:
parameters = {'depth'         : [6,8,10],
              'learning_rate' : [0.01, 0.05, 0.1,1],
              'iterations'    : [10000]}

In [ ]:
#https://setscholars.net/2019/02/19/how-to-find-optimal-parameters-for-catboost-using-gridsearchcv-for-regression-in-python/
model = CatBoostRegressor(loss_function='MAE', boosting_type='Ordered',task_type='GPU')
parameters = {'depth'         : [6],
              'learning_rate' : [1],
              'iterations'    : [10000]}
grid = GridSearchCV(estimator=model, param_grid = parameters, scoring='neg_mean_absolute_error',cv = 2)
grid.fit(X_train, y_train)

In [10]:
print('SVR(kernel=rbf')
print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))
print("The best score across ALL searched params: ", grid.best_score_)
print("Test-set: mean absolute error : {:.2f}".format(mean_absolute_error(y_test, grid.predict(X_test))))

SVR(kernel=rbf
Best params:
{'depth': 6, 'iterations': 10000, 'learning_rate': 1}

Best cross-validation score: -2.00
Test-set score: -1.64
The best score across ALL searched params:  -1.9965746342232633
Test-set: mean absolute error : 1.64


In [11]:
print("\n========================================================")
print(" Results from Grid Search " )
print("========================================================")    
    
print("\n The best estimator across ALL searched params:\n", grid.best_estimator_)
    
print("\n The best score across ALL searched params:\n", grid.best_score_)
    
print("\n The best parameters across ALL searched params:\n", grid.best_params_)


 Results from Grid Search 

 The best estimator across ALL searched params:

 The best score across ALL searched params:
 -1.9965746342232633

 The best parameters across ALL searched params:
 {'depth': 6, 'iterations': 10000, 'learning_rate': 1}


# Erkenntnisse

# Model selection
 - Random forrest
 - Other GradientBoost (AdaBoost)

 - Feed Forward NN (with Karas)
 - Reccurent Neuronal Net: LSTM (with Karas)
 - Convolutional NN https://www.kaggle.com/michael422/spectrogram-convolution https://www.kaggle.com/michael422/spectrogram-convolution
 
 
 - CatBoost vs. Light GBM vs. XGBoost https://www.kdnuggets.com/2018/03/catboost-vs-light-gbm-vs-xgboost.html
 - CatBoost-GridSearch https://setscholars.net/2019/02/19/how-to-find-optimal-parameters-for-catboost-using-gridsearchcv-for-regression-in-python/